## Modules

In [46]:
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.display import display, clear_output
import seaborn as sns

## Read CSV

In [ ]:
# 1. Load the CSV file with a different encoding
csv_file = 'Accidentes.csv'  # Change this path to the actual location of your file
df = pd.read_csv(csv_file, encoding='ISO-8859-1')  # Or use encoding='latin1'